<a href="https://colab.research.google.com/github/hannahrfong/CSC713M/blob/main/Investigatory_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **CSC713M Investigatory Project**

# Importing Libraries

In [31]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
#import scipy.stats as sps

In [34]:
fie_df = pd.read_csv('fie.csv')

# Dataset Description

## Brief Description

The Family Income and Expenditure (FIE) dataset is derived from the 2015 Family Income and Expenditure Survey (FIES) conducted by the Philippine Statistics Authority (PSA). The dataset provides data on the income and expenditure of Filipino households which can be used to formulate governmental policies and programs to elevate the socio-economic conditions of Filipinos.

## Dataset File Structure

The dataset contains 41,544 samples and 60 features. Each sample represents one Filipino household.

In [ ]:
fie_df.shape

The list of features in the dataset and their corresponding data types can be generated by running the code below.

In [ ]:
fie_df.info()

# Data Cleaning

The FIE dataset will undergo data cleaning to identify noisy, incomplete, and inconsistent data that can degrade the performance of machine learning models to be used with the dataset.

Specifically, the following aspects of the dataset will be checked:

*   Multiple Representations
*   Incorrect Datatypes
*   Default Values
*   Missing Data
*   Duplicate Data
*   Inconsistent Format









## Multiple Representations

The `unique` function is used to check if there exist different representations in the categorical features of the dataset.

In [ ]:
for column in fie_df.select_dtypes(include="object"):
    print("'{}' unique values:\n".format(column), fie_df[column].unique())
    print("")

Upon inspection, it can be confirmed that there are no multiple representations in the dataset.

## Incorrect Datatype

The `info` function is used to validate the data types of the values in the dataset.

In [ ]:
fie_df.info()

Given that the categorical features are an object type and the numerical features have an integer (`int64`) type, no incorrect datatypes were found amongst all the features.

## Default Values

The `unique` function is used to identify default values in the dataset and to assess if these are appropriate in the context of the project.

In [ ]:
for column in fie_df:
    print("'{}' unique values:\n".format(column), fie_df[column].unique())
    print("")

The result shows that the `Agricultural Household indicator` feature has 3 unique values (i.e. `0`, `1`, `2`). However, the metadata of the dataset specifies that there are only 2 possible values for this feature:
*   1 - Agricultural Household
*   2 - Non-agricultural Household


In [ ]:
fie_df.loc[fie_df['Agricultural Household indicator'] == 0].shape

(28106, 60)

There are 28,106 observations with the undefined `0` value for the `Agricultural Household indicator` feature, which is more than half (67.65%) of the dataset. Therefore, these observations cannot just be deleted.

Instead, the `Agricultural Household indicator` feature will be removed to handle the ambiguous data and prevent it from impairing the performance of the machine learning models to be used.

**!!! TODO: Can get correct value from raw dataset**

In [ ]:
fie_df = fie_df.drop(['Agricultural Household indicator'], axis=1)
fie_df.shape

(41544, 59)

## Missing Data

The `isnull` and `any` functions are used to check for missing values in the dataset.

In [ ]:
fie_df.isnull().any()

In [ ]:
nan_variables = fie_df.columns[fie_df.isnull().any()].tolist()
print(nan_variables)

['Household Head Occupation', 'Household Head Class of Worker']


In [ ]:
fie_df[nan_variables].isnull().sum()

Household Head Occupation         7536
Household Head Class of Worker    7536
dtype: int64

There are 2 features with missing values:

1. Household Head Occupation
2. Household Head Class of Worker

Let us check if these null values are missing at random or not by counterchecking with the `Household Head Job or Business Indicator` feature that indicates whether a household head is employed or not.



In [ ]:
job_variables = nan_variables
job_variables.append('Household Head Job or Business Indicator')

fie_df[job_variables].loc[(fie_df['Household Head Occupation'].isnull()) & (fie_df['Household Head Class of Worker'].isnull()) & (fie_df['Household Head Job or Business Indicator'] == 'No Job/Business')]

,Household Head Occupation,Household Head Class of Worker,Household Head Job or Business Indicator
8,NaN,NaN,No Job/Business
13,NaN,NaN,No Job/Business
14,NaN,NaN,No Job/Business
15,NaN,NaN,No Job/Business
26,NaN,NaN,No Job/Business
...,...,...,...
41520,NaN,NaN,No Job/Business
41529,NaN,NaN,No Job/Business
41533,NaN,NaN,No Job/Business
41535,NaN,NaN,No Job/Business


The executed code above returns observations that have a `NaN` value for `Household Head Occupation`, a `Nan` value for `Household Head of Class of Worker`, and a `No Job/Business` value for `Household Head Job or Business Indicator`. The result contains 7,536 observations which is the same number of observations that have null values for the 2 `nan_variables` (i.e. `Household Head Occupation` and `Household Head of Class of Worker`).

Based on this, it can be deduced that the missing values are not missing at random. Instead, they indicate the unemployment of household heads.

The null values will be handled by replacing them with an `Unemployed` value to properly label the data.

In [ ]:
fie_df = fie_df.fillna('Unemployed')
nan_variables = fie_df.columns[fie_df.isnull().any()].tolist()
print(nan_variables)

[]


After running the code above, there are no more missing values in the dataset.

## Duplicate Data

The `drop_duplicates` function is used to delete duplicate data in the dataset.

In [ ]:
fie_df.drop_duplicates()

There are 41,544 observations returned after running the `drop_duplicates` function, which is the same as the number of observations in the original dataset. Therefore, there are no duplicates in the dataset.

## Inconsistent Format

The Philippine regions have an inconsistent format, particularly for regions IX – Zamboanga Peninsula, XIII – Caraga, and ARMM. Region IX contains an misspelling of the region, Region XIII does not have its roman numeral as part of its label, and ARMM contains a whitespace character before the abbreviation.

In [ ]:
for column in fie_df:
    print("'{}' unique values:\n".format(column), fie_df[column].unique())
    print("")

In [ ]:
regions = {
    'IX - Zasmboanga Peninsula': 'IX - Zamboanga Peninsula',
    ' ARMM': 'ARMM',
    'Caraga': 'XIII - Caraga'
}

fie_df['Region'] = fie_df['Region'].map(regions).fillna(fie_df['Region'])

With this, we map these errors to the correct labels by using the `map()` function and the `regions` dictionary.

The `Type of Walls` feature has a subset of samples that contain a `NOt applicable` value. To make this consistent with other columns that have a `Not Applicable` value, we replace all current instances with the new value.

In [ ]:
fie_df['Type of Walls'] = fie_df['Type of Walls'].replace('NOt applicable', 'Not Applicable')

The `Total Income from Entrepreneurial Acitivites` feature misspells the word *Activities*.

In [ ]:
fie_df.rename(columns={'Total Income from Entrepreneurial Acitivites': 'Total Income from Entrepreneurial Activites'})

# Data Preprocessing

## Binning

### Total Household Income

**TODO: CITE PAPER ON THE BINS USED**

In [ ]:
bins = [0, 10957, 21914, 43828, 76669, 131484, 219140, fie_df['Total Household Income'].max() + 1]
labels=["Poor", "Low-income", "Lower middle", "Middle", "Upper middle", "Upper middle but not rich", "Rich"]

fie_df['Income Bracket'] = pd.cut(fie_df['Total Household Income'], bins=bins, labels=labels, right=False)
fie_df['Income Bracket'] = fie_df['Income Bracket'].astype('object')

### Household Head Highest Grade Completed

Based on the “Household Head Highest Grade Completed”, there are multiple labels that dictate a specific educational attainment that a person has completed. To simplify the labels, we bin these values into five categories: `No Grade Completed`, `Preschool`, `Primary Level`, `Secondary Level`, and `Tertiary Level`.

In [ ]:
grade = {
    'Preschool': 'Preschool',
    'No Grade Completed': 'No Grade Completed',
    'Grade 1': 'Primary Level',
    'Grade 2': 'Primary Level',
    'Grade 3': 'Primary Level',
    'Grade 4': 'Primary Level',
    'Grade 5': 'Primary Level',
    'Grade 6': 'Primary Level',
    'Elementary Graduate': 'Primary Level',
    'First Year High School': 'Secondary Level',
    'Second Year High School': 'Secondary Level',
    'Third Year High School': 'Secondary Level',
    'High School Graduate': 'Secondary Level',
    'Post Baccalaureate': 'Post Baccalaureate'
}

fie_df['Household Head Highest Grade Completed'] = fie_df['Household Head Highest Grade Completed'].map(grade)
fie_df['Household Head Highest Grade Completed'] = fie_df['Household Head Highest Grade Completed'].replace(np.nan, 'Tertiary Level')

# Exploratory Data Analysis

## Demographics

In [ ]:
region_counts = fie_df['Region'].value_counts()

plt.bar(region_counts.index, region_counts.values)

plt.xlabel('Region')
plt.ylabel('Count')
plt.title('Number of Observations per Region')

plt.xticks(rotation=75)
plt.show()

In [ ]:
data = fie_df['Household Head Age']
mean_age = data.mean()

plt.hist(data, bins=50, edgecolor='w')

x_axis_labels = range(0, 101, 10)
plt.xticks(x_axis_labels)
plt.xlabel('Age')
plt.ylabel('Count')

plt.axvline(mean_age, color='red', linestyle='dashed', linewidth=2, label=f'Mean Age = {mean_age}')

plt.legend()
plt.show()

In [ ]:
fie_df.boxplot("Total Household Income", by="Household Head Class of Worker", figsize=(10,6), vert=False)
plt.show()

In [ ]:
fie_df.boxplot('Household Head Age', by='Region', vert=False)
plt.title('Household Head Age by Region')
plt.show()

In [ ]:
fie_df.boxplot('Total number of family members employed', by='Region', vert=False)
plt.title('Total number of family members employed by Region')
plt.show()

In [ ]:
hhs_counts = fie_df['Household Head Sex'].value_counts()
plt.figure(figsize=(10,6))
hhs_counts.plot(kind='bar', rot=0)
plt.xlabel('Household Head Sex')
plt.ylabel('Count')
plt.title('Household Head Sex')
plt.show()

In [ ]:
grouped = fie_df.groupby(['Region', 'Household Head Sex']).size().unstack(fill_value=0)

totals = grouped.sum(axis=1)
grouped_ratio = grouped.divide(totals, axis=0)
ax = grouped_ratio.plot(kind='bar', stacked=True, figsize=(10, 6))
ax.set_xlabel('Region')
ax.set_ylabel('Count')
ax.set_title('Household Head Sex Per Region')
plt.legend(title='Household Head Sex', loc='upper right')

plt.show()

## Income

In [ ]:
grouped = fie_df.groupby('Income Bracket')
summary_stats = grouped['Total Household Income'].describe()
summary_stats.reset_index(inplace=True)

summary_stats

In [ ]:
value_counts = fie_df['Income Bracket'].value_counts()

plt.barh(value_counts.index, value_counts.values)

plt.xlabel('Count')
plt.ylabel('Income Bracket')
plt.title('Income Distribution')

plt.show()

In [ ]:
fie_df.boxplot("Total Household Income", by="Region", figsize=(15,10), vert=False)
plt.title('Total Household Income by Region')
plt.show()

In [ ]:
# In the Philippines
country_counts = fie_df['Main Source of Income'].value_counts()
plt.figure(figsize=(10,6))
country_counts.plot(kind='bar', rot=0)
plt.xlabel('Source of Income')
plt.ylabel('Count')
plt.title('Main Sources of Income')
plt.show()

In [ ]:
# Per region
grouped = fie_df.groupby(['Region', 'Main Source of Income']).size().unstack(fill_value=0)
region_totals = grouped.sum(axis=1)
grouped_ratio = grouped.divide(region_totals, axis=0)
ax = grouped_ratio.plot(kind='bar', stacked=True, figsize=(10, 6))
ax.set_xlabel('Region')
ax.set_ylabel('Ratio')
ax.set_title('Ratio of Main Source of Income by Region')
plt.legend(title='Main Source of Income', loc='upper right')

plt.show()

In [ ]:
# Per income group
grouped = fie_df.groupby(['Income Bracket', 'Household Head Highest Grade Completed']).size().unstack(fill_value=0)

educ_totals = grouped.sum(axis=1)
grouped_ratio = grouped.divide(educ_totals, axis=0)
ax = grouped_ratio.plot(kind='bar', stacked=True, figsize=(10, 6))
ax.set_xlabel('Income Bracket')
ax.set_ylabel('Ratio')
ax.set_title('Ratio of Educational Attainment by Income Bracket')
plt.legend(title='Educational Attainment', loc='upper right')

plt.show()

## Expenses

In [ ]:
fie_df.plot.scatter(x='Total Household Income', y='Total Food Expenditure', alpha=0.5)
plt.title('Relationship of Total Household Income and Total Food Expenditure')

In [ ]:
ratio = fie_df['Total Food Expenditure'] / fie_df['Total Household Income']

plt.scatter(x=fie_df['Total Household Income'], y=ratio, alpha=0.5)

plt.title('Relationship of Total Household Income and Ratio of Food Expenditure to Income')
plt.xlabel('Total Household Income')
plt.ylabel('Ratio of Food Expenditure to Income')

plt.show()

In [ ]:
other_exp = ['Clothing, Footwear and Other Wear Expenditure', 'Housing and water Expenditure',
       'Imputed House Rental Value', 'Medical Care Expenditure', 'Transportation Expenditure', 'Communication Expenditure',
       'Education Expenditure', 'Miscellaneous Goods and Services Expenditure', 'Restaurant and hotels Expenditure', 'Special Occasions Expenditure', 'Crop Farming and Gardening expenses']

expense_scatter = fie_df.groupby("Region")[other_exp].mean().round(2).sort_values(by=other_exp, ascending=False)

ax = expense_scatter.plot(kind='bar', rot=70, stacked=True, figsize=(10,6))
ax.set_xlabel('Region')
ax.set_ylabel('Other Expenses Amount')
ax.set_title('Other Expenses per Region')
plt.legend(title='Other Expenses', loc='upper right')

plt.show()

# Feature Selection

## One Hot Encoding of Categorical Variables

In [ ]:
from sklearn.preprocessing import OneHotEncoder

In [ ]:
fie_df.head()
fie_df.tail()

In [ ]:
fie_df.dtypes

In [ ]:
ohe = OneHotEncoder()

In [ ]:
categorical_columns = fie_df.select_dtypes(include=['object']).columns
categorical_columns = categorical_columns[categorical_columns != 'Income Bracket']

In [ ]:
feature_array = ohe.fit_transform(fie_df[categorical_columns]).toarray()

In [ ]:
ohe.categories_

In [ ]:
feature_labels = ohe.categories_

In [ ]:
feature_labels = np.array(feature_labels, dtype='object').ravel()
flattened_feature_labels = np.concatenate(feature_labels)

In [ ]:
print(flattened_feature_labels)

In [ ]:
features = pd.DataFrame(feature_array, columns = flattened_feature_labels)

In [ ]:
features.head()

In [ ]:
numerical_columns = fie_df.select_dtypes(include=['int64'])

new_fie_df = pd.concat([features, numerical_columns], axis=1)

## Label Encoding of Target Variable

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
label_encoder = LabelEncoder()

custom_labels = ["Poor", "Low-income", "Lower middle", "Middle", "Upper middle", "Upper middle but not rich", "Rich"]

# Fit the label encoder with the custom mapping
label_encoder.fit(custom_labels)

# Create a custom mapping dictionary
custom_mapping = {label: idx for idx, label in enumerate(custom_labels)}

# Transform your data using the custom mapping
new_fie_df['Income Bracket'] = fie_df['Income Bracket'].map(custom_mapping)

## Correlation

In [ ]:
corr = new_fie_df.corr()
correlation_threshold = 0.5  # Adjust this threshold as needed

# Calculate the absolute correlation matrix
corr_matrix = new_fie_df.corr().abs()

# Create an upper triangular mask to avoid duplicates and the diagonal
upper_triangular = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))

# Find the columns (features) to drop based on the threshold
to_drop = [column for column in upper_triangular.columns if any(upper_triangular[column] > correlation_threshold)]

# Create a new DataFrame with the highly correlated features removed
df_filtered = new_fie_df.drop(columns=to_drop)

In [ ]:
df_filtered.shape

## Backward Elimination Method

In [ ]:
from mlxtend.feature_selection import SequentialFeatureSelector

In [ ]:
X = new_fie_df.iloc[:,:-1]
y = new_fie_df.iloc[:,-1]

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
backward_feature_selector = SequentialFeatureSelector(RandomForestClassifier(n_jobs=-1),
                                                      k_features=(1, 454),
                                                      forward=False,
                                                      floating=False,
                                                      verbose=2,
                                                      scoring='accuracy',
                                                      cv=5).fit(X_train, y_train)

In [ ]:
backward_feature_selector.k_feature_names_

In [ ]:
backward_feature_selector.k_score_

# K-Means

In [ ]:
fie_df.shape

In [ ]:
new_fie_df.shape

In [ ]:
df_filtered.shape

In [ ]:
fie_df['Income Bracket']

In [ ]:
bins = [0, 10957, 21914, 43828, 76669, 131484, 219140, df_filtered['Total Household Income'].max() + 1]
labels=["Poor", "Low-income", "Lower middle", "Middle", "Upper middle", "Upper middle but not rich", "Rich"]

df_filtered['Income Bracket'] = pd.cut(df_filtered['Total Household Income'], bins=bins, labels=labels, right=False)
df_filtered['Income Bracket'] = df_filtered['Income Bracket'].astype('object')

custom_labels = ["Poor", "Low-income", "Lower middle", "Middle", "Upper middle", "Upper middle but not rich", "Rich"]

# Create a custom mapping dictionary
custom_mapping = {label: idx for idx, label in enumerate(custom_labels)}

# Transform your data using the custom mapping
df_filtered['Income Bracket'] = df_filtered['Income Bracket'].map(custom_mapping)

In [ ]:
fie_df_num = fie_df.select_dtypes(['int64'])
new_fie_df_num = new_fie_df.select_dtypes(['float64', 'int64'])
df_filtered_num = df_filtered.select_dtypes(['float64', 'int64'])

In [ ]:
from sklearn.cluster import KMeans

In [ ]:
from sklearn.cluster import KMeans
fie_kmeans = KMeans(7, init='random')
fie_kmeans = fie_kmeans.fit(fie_df_num)
fie_df['Predicted Class'] = pd.Series(fie_kmeans.predict(fie_df_num))
fie_df['Predicted Class'].value_counts()

In [ ]:
new_fie_kmeans = KMeans(7, init='random')
new_fie_kmeans = new_fie_kmeans.fit(new_fie_df_num)
new_fie_df['Predicted Class'] = pd.Series(new_fie_kmeans.predict(new_fie_df_num))
new_fie_df['Predicted Class'].value_counts()

In [ ]:
df_filtered_kmeans = KMeans(7, init='random')
df_filtered_kmeans = df_filtered_kmeans.fit(df_filtered_num)
df_filtered['Predicted Class'] = pd.Series(df_filtered_kmeans.predict(df_filtered_num))
df_filtered['Predicted Class'].value_counts()

In [ ]:
# tp_fie_df = fie_df[['Income Bracket', 'Predicted Class']]
# tp_new_fie_df = new_fie_df[['Income Bracket', 'Predicted Class']]
tp_filtered_df = df_filtered[['Income Bracket', 'Predicted Class']]

# tp_fie_df['Bracket and Class'] = tp_fie_df[['Income Bracket', 'Predicted Class']].apply(tuple, axis=1)
# tp_new_fie_df['Bracket and Class'] = tp_new_fie_df[['Income Bracket', 'Predicted Class']].apply(tuple, axis=1)
tp_filtered_df['Bracket and Class'] = tp_filtered_df[['Income Bracket', 'Predicted Class']].apply(tuple, axis=1)

In [ ]:
fie_counts = tp_fie_df['Bracket and Class'].value_counts()
plt.figure(figsize=(10,6))
fie_counts.plot(kind='bar', rot=90)
plt.xlabel('(Income Bracket, Predicted Class)')
plt.ylabel('Count')
plt.title('Income Bracket and Predicted Class - Original Data')
plt.show()

In [ ]:
new_fie_counts = tp_new_fie_df['Bracket and Class'].value_counts()
plt.figure(figsize=(10,6))
new_fie_counts.plot(kind='bar', rot=90)
plt.xlabel('(Income Bracket, Predicted Class)')
plt.ylabel('Count')
plt.title('Income Bracket and Predicted Class - One Hot Encoding Applied ')
plt.show()

In [ ]:
filtered_counts = tp_filtered_df['Bracket and Class'].value_counts()
plt.figure(figsize=(10,6))
filtered_counts.plot(kind='bar', rot=90)
plt.xlabel('(Income Bracket, Predicted Class)')
plt.ylabel('Count')
plt.title('Income Bracket and Predicted Class - One Hot Encoding and Correlation Selection Applied ')
plt.show()

# Testing Stuff

In [ ]:
from sklearn.cluster import KMeans

In [ ]:
test_fie_df = new_fie_df

In [ ]:
# km = KMeans(6, init='random')
# km = km.fit(test_fie_df)
# test_fie_df['Predicted Class'] = pd.Series(km.predict(test_fie_df))
# test_fie_df['Predicted Class'].value_counts()

vars_fie = test_fie_df.groupby('Predicted Class').mean().var(axis=0)
print(test_fie_df.shape)

ALPHA = 1e-5
num_dropped_cols = 1
dropped_cols = []

while dropped_cols != 0:
  km = KMeans(6, init='random')
  km = km.fit(test_fie_df)
  test_fie_df['Predicted Class'] = pd.Series(km.predict(test_fie_df))
  test_fie_df['Predicted Class'].value_counts()

  vars_fie = test_fie_df.groupby('Predicted Class').mean().var(axis=0)

  # ERROR: vars_fie has multiple indices named "Others"
  # Caused by original columns have "Others" as one of its unique values
  # that when it's one-hot encoded, it creates multiple columns named
  # "Others" as well, affecting column dropping for this algorithm
  for ind in vars_fie.index:
    if vars_fie[ind] < ALPHA:
      dropped_cols.append(ind)

  num_dropped_cols = len(dropped_cols)
  test_fie_df = test_fie_df.drop(labels=dropped_cols, axis=1)

print(test_fie_df.shape)

In [ ]:
km = KMeans(6, init='random')
km = km.fit(test_fie_df)
test_fie_df['Predicted Class'] = pd.Series(km.predict(test_fie_df))
test_fie_df['Predicted Class'].value_counts()

vars_fie = test_fie_df.groupby('Predicted Class').mean().var(axis=0)

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [ ]:
test_vars_fie = vars_fie

In [ ]:
from google.colab import drive
drive.mount('drive')

In [ ]:
test_vars_fie.to_csv('drive/My Drive/vars_fie.csv')

Drive already mounted at drive; to attempt to forcibly remount, call drive.mount("drive", force_remount=True).


In [ ]:
from google.colab import files
files.download("vars_fie.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
new_fie_df['Income Bracket'].value_counts()

6    14983
5    10980
4    10604
3     4015
2      889
1       73
Name: Income Bracket, dtype: int64

In [ ]:
km = KMeans(6, init='random')
km = km.fit(test_fie_df)
test_fie_df['Predicted Class'] = pd.Series(km.predict(test_fie_df))
test_fie_df['Predicted Class'].value_counts()

vars_fie = test_fie_df.groupby('Predicted Class').mean().var(axis=0)

for ind in vars_fie.index:
  if vars_fie[ind] < ALPHA:
    dropped_cols.append(ind)

num_dropped_cols = len(dropped_cols)
test_fie_df = test_fie_df.drop(labels=dropped_cols, axis=1)

print(test_fie_df.shape)

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


ValueError: ignored

In [ ]:
for ind in vars_fie.index:
  print(type(vars_fie[ind]))